In [2]:
!pip install llama-index --quiet
!pip install neo4j --quiet
!pip install llama-index-graph-stores-neo4j --quiet
!pip install llama-parse --quiet
!pip install qdrant_client --quiet
!pip install llama-index-vector-stores-qdrant --quiet
!pip install llama-index-embeddings-fastembed --quiet
!pip install llama-index-llms-groq --quiet
!pip install aiofiles --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.4/187.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 

In [1]:
# HuggingFace Data Preprocessing Script
########################################

import os
import re
import sys
import ast
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import drive

# Set numpy and pandas display options
np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Mount Google Drive
drive.mount('/content/drive')

# Load the dataset
df = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/caselaw_rag_data/opinions_2015_before_2024.csv')
dataset = df.copy()

# Extract multiple attorney IDs
def extract_attorneys_ids(text):
    if not isinstance(text, str):
        return None
    attorneys_pattern = r'<attorneys id="(.*?)">'
    attorneys_ids = re.findall(attorneys_pattern, text)
    return ', '.join(attorneys_ids) if attorneys_ids else None

dataset['attorneys_id'] = dataset['headmatter'].apply(extract_attorneys_ids)

def remove_specific_xml_tags(text):
    """
    Remove specific XML tags from the text while retaining certain attributes.

    Args:
        text (str): The text containing XML tags.

    Returns:
        str: Text with specified XML tags removed.
    """
    if isinstance(text, str):
        # Retain specified opening tags with their attributes and remove angle brackets
        text = re.sub(r'<(parties|docketnumber|decisiondate) id="([^"]*)">', r'\1 id="\2"', text)
        text = re.sub(r'<attorneys id="([^"]*)">', r'attorneys id="\1"', text)
        text = re.sub(r'<span citation-index="([^"]*)" class="([^"]*)" label="([^"]*)">', r'span citation-index="\1" class="\2" label="\3"', text)
        # Remove specific closing tags
        text = re.sub(r'</(attorneys|court|decisiondate|docketnumber|parties|span)>', '', text)
        # Remove all other tags
        text = re.sub(r'<.*?>', '', text)
        # Normalize spaces but keep newlines intact
        text = re.sub(r'[ ]{2,}', ' ', text)  # Replace multiple spaces with a single space
        return text.strip()  # Clean up any leading/trailing whitespace
    return text  # Return unchanged if not a string

# Clean the 'headmatter' column by removing specified tags
dataset['headmatter'] = dataset['headmatter'].apply(remove_specific_xml_tags)

Mounted at /content/drive


In [3]:
dataset.head()

,id,judges,date_filed,date_filed_is_approximate,slug,case_name_short,case_name,case_name_full,attorneys,nature_of_suit,posture,syllabus,headnotes,summary,disposition,history,other_dates,cross_reference,correction,citation_count,precedential_status,arguments,citations,opinions,court_short_name,court_full_name,court_type,court_jurisdiction,opinion_id,type,per_curiam,author_id,author_str,opinion_text,parties,docketnumber,court,otherdate,decisiondate,seealso,attorneys_headmatter,judges_headmatter,p,citation,headmatter,num_opinions,0_0,1_1,2_2,3_3,4_4,5_5,6_6,7_7,8_8,year_filed,attorneys_id
0,7311944,Spatt,2015-01-02,False,placide-eugene-v-visiting-nurse-service,Placide-Eugene,Placide-Eugene v. Visiting Nurse Service,Marie PLACIDE-EUGENE v. VISITING NURSE SERVICE...,"White Ricotta & Marks, P.C., by: Thomas Ricott...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Published,NaN,['86 F. Supp. 3d 132'],"[{'author_str': 'Spatt', 'per_curiam': False, ...",E.D. New York,"District Court, E.D. New York",FD,"New York, NY",7229848,020lead,False,NaN,Spatt,"\nMEMORANDUM OF DECISION AND ORDER\nSPATT, Dis...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},1,"{'author_str': 'Spatt', 'per_curiam': False, '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,None
1,7311182,Hillman,2015-01-02,False,thirkield-v-neary-hunter-obgyn-llc,Thirkield,"Thirkield v. Neary & Hunter Ob/Gyn, LLC","Kimberly THIRKIELD v. NEARY & HUNTER OB/GYN, L...","Michael O. Shea, Law Office of Michael Q_ ^ Wi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Published,NaN,['76 F. Supp. 3d 339'],"[{'author_str': 'Hillman', 'per_curiam': False...",D. Massachusetts,"District Court, D. Massachusetts",FD,"Massachusetts, MA",7229086,020lead,False,NaN,Hillman,\n*342ORDER AND MEMORANDUM OF DECISION ON DEFE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},1,"{'author_str': 'Hillman', 'per_curiam': False,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,None
2,7311845,Smith,2015-01-07,False,keeton-v-big-lots-stores-inc,Keeton,"Keeton v. Big Lots Stores, Inc.","Sondra KEETON v. BIG LOTS STORES, INC.","Kimberly R. Dodson, Law Offices of Kimberly R....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Published,NaN,['84 F. Supp. 3d 1290'],"[{'author_str': 'Smith', 'per_curiam': False, ...",N.D. Alabama,"District Court, N.D. Alabama",FD,"Alabama, AL",7229749,020lead,False,NaN,Smith,"\nMEMORANDUM OPINION AND ORDER\nLYNWOOD SMITH,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},1,"{'author_str': 'Smith', 'per_curiam': False, '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,None
3,7311254,"Kollar, Kotelly",2015-01-07,False,king-v-holder,King,King v. Holder,"Willard T. KING, Jr. v. Eric Himpton HOLDER, Jr.","Donald B. Terrell, Donald B. Terrell Associate...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,Published,NaN,['77 F. Supp. 3d 146'],"[{'author_str': 'Kollarkotelly', 'per_curiam':...",District of Columbia,"District Court, District of Columbia",FD,"USA, Federal",7229158,020lead,False,NaN,Kollarkotelly,"\nMEMORANDUM OPINION\n(January 7, 2015)\nCOLLE...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},1,"{'author_str': 'Kollarkotelly', 'per_curiam': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,None
4,7311460,Dubose,2015-01-07,False,brown-v-mobile-county-commissioners,Brown,Brown v. Mobile County Commissioners,Karen Padgett BROWN v. MOBILE COUNTY COMMISSIO...,"Karen Padgett Brown, Mobile, AL, pro se., R. S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Signed Jan. 6, 2015.",NaN,NaN,0,Published,NaN,['79 F. Supp. 3d 1259'],"[{'author_str': 'Dubose', 'per_curiam': False,...",S.D. Alabama,"District Court, S.D. Alabama",FD,"Alabama, AL",7229364,020lead,False,NaN,Dubose,"\nORDER\nKRISTI K. DuBOSE, District Judge.\nTh...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},1,"{'author_str': 'Dubose', 'per_curiam': False, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,None


In [4]:
aggregated_df = dataset.copy()

def safe_clean_text(text):
    try:
        if pd.isna(text):
            return ""
        if not isinstance(text, str):
            text = str(text)
        # Remove curly braces, square brackets, and single quotes
        text = re.sub(r'[\{\}\[\]\']', '', text)
        # Remove any leading/trailing whitespace
        text = text.strip()
        # Replace multiple spaces with a single space
        text = re.sub(r'\s+', ' ', text)
        return text
    except Exception as e:
        return ""

def safe_apply_clean_text(series):
    return series.apply(safe_clean_text)

# Applying safe_clean_text to individual columns before concatenation
columns_to_clean = ['headmatter', 'date_filed', 'author_str', 'opinion_id', 'type', 'opinion_text']

for col in columns_to_clean:
    if col in aggregated_df.columns:
        aggregated_df[col] = safe_apply_clean_text(aggregated_df[col])

# Function to concatenate fields, skipping empty ones
def concatenate_fields(row):
    fields = []
    if row['headmatter']:
        fields.append(row['headmatter'])
    fields.extend([
        f"Date Filed: {row['date_filed']}",
        f"Author: {row['author_str']}",
        f"Opinion ID: {row['opinion_id']}",
        f"Type: {row['type']}",
        f"Opinion Text:\n{row['opinion_text']}"
    ])
    return '\n'.join(filter(None, fields))

# Apply the concatenation function
aggregated_df['case_opinions'] = aggregated_df.apply(concatenate_fields, axis=1)

# Reset index and select only 'id' and 'case_opinions' columns
aggregated_df = aggregated_df[['id', 'case_opinions']].reset_index(drop=True)

In [5]:
aggregated_df.sort_values(by='id', ascending=True).head()

,id,case_opinions
9,2770074,"citation: 2 NYS3d 132, parties: Jeffrey Chiara..."
33,2777197,"parties: Alyce T. CONLON, Plaintiff-Appellant,..."
34,2777197,"parties: Alyce T. CONLON, Plaintiff-Appellant,..."
39,2778493,"citation: 107 A.3d 1250, parties: ILDA AGUAS, ..."
38,2778493,"citation: 107 A.3d 1250, parties: ILDA AGUAS, ..."


In [6]:
print(aggregated_df['case_opinions'][34])

parties: Alyce T. CONLON, Plaintiff-Appellant, v. INTERVARSITY CHRISTIAN FELLOWSHIP/USA, Fred Bailey, And Marc Papai, Defendants-Appellees., docketnumber: No. 14-1549., court: United States Court of Appeals, Sixth Circuit., otherdate: Argued: Dec. 3, 2014., decisiondate: Decided and Filed: Feb. 5, 2015., judges: BATCHELDER, J., delivered the opinion of the court in which BECKWITH, D.J., joined, and ROGERS, J., joined in part. ROGERS, J. (pp. 837-38), delivered a separate opinion concurring in parts I, II, III.E, and the result.
Date Filed: 2015-02-05
Author: Rogers
Opinion ID: 9805954
Type: 030concurrence
Opinion Text:
ROGERS, Circuit Judge, concurring. I join parts I, II and III.E of the majority opinion, and concur in the result. Essentially for the reasons given in parts III.A and B of the majority opinion, IVCF can assert the ministerial exception, and that exception protects against the claim asserted by Conlon. Conlon’s sole claim on appeal is under Title VII. Under the Constitut

In [7]:
print(aggregated_df['case_opinions'][33])

parties: Alyce T. CONLON, Plaintiff-Appellant, v. INTERVARSITY CHRISTIAN FELLOWSHIP/USA, Fred Bailey, And Marc Papai, Defendants-Appellees., docketnumber: No. 14-1549., court: United States Court of Appeals, Sixth Circuit., otherdate: Argued: Dec. 3, 2014., decisiondate: Decided and Filed: Feb. 5, 2015., judges: BATCHELDER, J., delivered the opinion of the court in which BECKWITH, D.J., joined, and ROGERS, J., joined in part. ROGERS, J. (pp. 837-38), delivered a separate opinion concurring in parts I, II, III.E, and the result.
Date Filed: 2015-02-05
Author: Batchelder
Opinion ID: 9805953
Type: 020lead
Opinion Text:
OPINION ALICE M. BATCHELDER, Circuit Judge. Alyce Conlon worked at InterVarsity Christian Fellowship/USA (“IVCF”) in Michigan as a spiritual director, involved in providing religious counsel and prayer. She informed IVCF that she was contemplating divorce, at which point IVCF put her on paid — and later unpaid — leave. When her marital situation continued to worsen despite 

In [8]:
dataset['id'].isna().sum(), aggregated_df['id'].isna().sum()

(0, 0)

In [9]:
dataset['id'].nunique(), aggregated_df['id'].nunique()

(1230, 1230)

In [10]:
column_stats = pd.DataFrame()
for col in dataset.columns:
  # if int((dataset[col].isna().sum()/len(dataset))*100) < 95:
  column_stats = pd.concat(
      [column_stats,
       pd.DataFrame([[col, int(dataset[col].isna().sum()),
                      str(int((dataset[col].isna().sum()/len(dataset))*100))+'%',
                      int(dataset[col].notna().sum()),
                      str(int((dataset[col].notna().sum()/len(dataset))*100))+'%']],
                           columns=['Column Name', 'Null Values', 'Null %', 'Not Null Values', 'Not Null Value %'])], ignore_index=True
      )

pd.DataFrame(column_stats).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56
Column Name,id,judges,date_filed,date_filed_is_approximate,slug,case_name_short,case_name,case_name_full,attorneys,nature_of_suit,posture,syllabus,headnotes,summary,disposition,history,other_dates,cross_reference,correction,citation_count,precedential_status,arguments,citations,opinions,court_short_name,court_full_name,court_type,court_jurisdiction,opinion_id,type,per_curiam,author_id,author_str,opinion_text,parties,docketnumber,court,otherdate,decisiondate,seealso,attorneys_headmatter,judges_headmatter,p,citation,headmatter,num_opinions,0_0,1_1,2_2,3_3,4_4,5_5,6_6,7_7,8_8,year_filed,attorneys_id
Null Values,0,19,0,0,0,248,0,0,22,1264,1312,1308,1312,1312,1312,1310,1087,1303,1312,0,0,1307,0,0,0,0,0,0,0,0,0,1312,70,0,1165,1182,1178,1239,1167,1312,1203,1218,1294,1293,0,0,0,1141,1159,1275,1308,1312,1312,1312,1312,0,1312
Null %,0%,1%,0%,0%,0%,18%,0%,0%,1%,96%,100%,99%,100%,100%,100%,99%,82%,99%,100%,0%,0%,99%,0%,0%,0%,0%,0%,0%,0%,0%,0%,100%,5%,0%,88%,90%,89%,94%,88%,100%,91%,92%,98%,98%,0%,0%,0%,86%,88%,97%,99%,100%,100%,100%,100%,0%,100%
Not Null Values,1312,1293,1312,1312,1312,1064,1312,1312,1290,48,0,4,0,0,0,2,225,9,0,1312,1312,5,1312,1312,1312,1312,1312,1312,1312,1312,1312,0,1242,1312,147,130,134,73,145,0,109,94,18,19,1312,1312,1312,171,153,37,4,0,0,0,0,1312,0
Not Null Value %,100%,98%,100%,100%,100%,81%,100%,100%,98%,3%,0%,0%,0%,0%,0%,0%,17%,0%,0%,100%,100%,0%,100%,100%,100%,100%,100%,100%,100%,100%,100%,0%,94%,100%,11%,9%,10%,5%,11%,0%,8%,7%,1%,1%,100%,100%,100%,13%,11%,2%,0%,0%,0%,0%,0%,100%,0%


In [11]:
dataset.drop(['headmatter', 'opinion_text', '0_0',	'1_1',	'2_2',	'3_3',	'4_4',	'5_5',	'6_6',	'7_7',	'8_8',	'year_filed'], axis=1, inplace=True)

In [12]:
node_columns = ['id', 'judges', 'author_id', 'author_str', 'court_type', 'court_jurisdiction', 'case_name_full', 'slug', 'date_filed', 'decisiondate', 'nature_of_suit',
                'posture', 'disposition', 'citation_count', 'precedential_status', 'attorneys', 'case_name', 'citations', 'opinion_id', 'type', 'docketnumber']

# Drop columns not needed for graph nodes
df_nodes = dataset[node_columns].copy()

dataset_nodes = df_nodes.drop_duplicates(subset='id', keep='first')

In [ ]:
import pandas as pd
import numpy as np
from neo4j import AsyncGraphDatabase
import asyncio
from google.colab import userdata, drive
import logging
from typing import List
from qdrant_client import QdrantClient
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex, Document, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.llms.groq import Groq
from concurrent.futures import ThreadPoolExecutor


# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Mount Google Drive
drive.mount('/content/drive')

def safe_convert(v):
    if isinstance(v, (list, np.ndarray)):
        return [str(item) for item in v if pd.notna(item)]
    elif pd.isna(v):
        return None
    else:
        return str(v)

class AsyncNeo4jLoader:
    def __init__(self, uri, username, password):
        self.driver = AsyncGraphDatabase.driver(uri, auth=(username, password))

    async def close(self):
        await self.driver.close()

    async def test_connection(self):
        try:
            async with self.driver.session() as session:
                result = await session.run("RETURN 1 AS num")
                record = await result.single()
                assert record["num"] == 1
                logger.info("Successfully connected to Neo4j database")
        except Exception as e:
            logger.error(f"Failed to connect to Neo4j database: {str(e)}")
            raise

    async def clear_database(self):
        query = "MATCH (n) DETACH DELETE n"
        async with self.driver.session() as session:
            await session.run(query)
        logger.info("Database cleared")

    async def create_constraints(self):
        constraints = [
            "CREATE CONSTRAINT IF NOT EXISTS FOR (n:Case) REQUIRE n.id IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (n:Judge) REQUIRE n.id IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (n:Attorney) REQUIRE n.id IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (n:Court) REQUIRE n.id IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (n:Party) REQUIRE n.id IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (n:Citation) REQUIRE n.id IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (n:Opinion) REQUIRE n.id IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (n:Docket) REQUIRE n.id IS UNIQUE"
        ]
        async with self.driver.session() as session:
            for constraint in constraints:
                await session.run(constraint)
        logger.info("Constraints created")

    async def load_data(self, df):
        async with self.driver.session() as session:
            for i, row in df.iterrows():
                try:
                    logger.info(f"Processing row {i}")
                    await self._create_case_graph(session, row)
                except Exception as e:
                    logger.error(f"Error processing row {i}: {str(e)}")
                    logger.error(f"Problematic row data: {row.to_dict()}")
        logger.info(f"Loaded {len(df)} cases")

    async def _create_case_graph(self, session, row):
        # Create Case node
        case_query = """
        MERGE (c:Case {id: $id})
        SET c.case_name = $case_name,
            c.date_filed = $date_filed,
            c.decisiondate = $decisiondate,
            c.slug = $slug,
            c.nature_of_suit = $nature_of_suit,
            c.posture = $posture,
            c.disposition = $disposition,
            c.citation_count = $citation_count,
            c.precedential_status = $precedential_status,
            c.case_name_full = $case_name_full
        """
        case_params = {k: safe_convert(v) for k, v in dict(row).items()}

        try:
            await session.run(case_query, case_params)

        except Exception as e:
            logger.error(f"Error processing row {row['id']}: {str(e)}")
            logger.error(f"Problematic query: {case_query}")
            logger.error(f"Problematic parameters: {case_params}")
            raise

        # Create Judge nodes and relationships
        if isinstance(row['judges'], str) and row['judges']:
            judges = [judge.strip() for judge in row['judges'].split(',')]
            judge_query = """
            UNWIND $judges AS judge
            MERGE (j:Judge {name: judge})
            WITH j
            MATCH (c:Case {id: $id})
            MERGE (c)-[:DECIDED_BY]->(j)
            """
            await session.run(judge_query, {'id': str(row['id']), 'judges': judges})

        # Create Author (Judge) node and relationship
        if pd.notna(row['author_id']) and pd.notna(row['author_str']):
            author_query = """
            MERGE (a:Judge {id: $author_id})
            SET a.name = $author_str
            WITH a
            MATCH (c:Case {id: $id})
            MERGE (c)-[:AUTHORED_BY]->(a)
            """
            await session.run(author_query, {
                'id': str(row['id']),
                'author_id': str(row['author_id']),
                'author_str': str(row['author_str'])
            })

        # Create Court node and relationship
        if pd.notna(row['court_type']) and pd.notna(row['court_jurisdiction']):
            court_query = """
            MERGE (ct:Court {id: $court_id})
            SET ct.type = $court_type,
                ct.jurisdiction = $court_jurisdiction
            WITH ct
            MATCH (c:Case {id: $id})
            MERGE (c)-[:HEARD_IN]->(ct)
            """
            await session.run(court_query, {
                'id': str(row['id']),
                'court_id': f"court_{row['id']}",
                'court_type': str(row['court_type']),
                'court_jurisdiction': str(row['court_jurisdiction'])
            })

        # Create Attorney nodes and relationships
        if pd.notna(row['attorneys']):
            attorneys = str(row['attorneys']).split(', ')
            attorney_query = """
            UNWIND $attorneys AS attorney
            MERGE (a:Attorney {name: attorney})
            WITH a
            MATCH (c:Case {id: $id})
            MERGE (c)-[:REPRESENTED_BY]->(a)
            """
            await session.run(attorney_query, {'id': str(row['id']), 'attorneys': attorneys})

        # Create Party nodes (Plaintiff and Defendant) and relationships
        if pd.notna(row['case_name']):
            parties = str(row['case_name']).split(' v. ')
            if len(parties) == 2:
                plaintiff, defendant = parties
                party_query = """
                MERGE (p:Party:Plaintiff {name: $plaintiff_name})
                MERGE (d:Party:Defendant {name: $defendant_name})
                WITH p, d
                MATCH (c:Case {id: $id})
                MERGE (p)-[:FILED_CASE]->(c)
                MERGE (c)-[:AGAINST]->(d)
                """
                await session.run(party_query, {
                    'id': str(row['id']),
                    'plaintiff_name': plaintiff,
                    'defendant_name': defendant
                })

        # Create Citation nodes and relationships
        citations = row['citations']
        if isinstance(citations, (list, np.ndarray)) and len(citations) > 0:
            citations = [str(c) for c in citations if pd.notna(c)]
            if citations:
                citation_query = """
                UNWIND $citations AS citation
                MERGE (cit:Citation {text: citation})
                WITH cit
                MATCH (c:Case {id: $id})
                MERGE (c)-[:CITED_BY]->(cit)
                """
                await session.run(citation_query, {'id': str(row['id']), 'citations': citations})

        # Create Opinion node and relationship
        if pd.notna(row['opinion_id']) and pd.notna(row['type']):
            opinion_query = """
            MERGE (o:Opinion {id: $opinion_id})
            SET o.type = $type
            WITH o
            MATCH (c:Case {id: $id})
            MERGE (c)-[:HAS_OPINION]->(o)
            """
            await session.run(opinion_query, {
                'id': str(row['id']),
                'opinion_id': str(row['opinion_id']),
                'type': str(row['type'])
            })

        # Create Docket node and relationship
        if 'docketnumber' in row and pd.notna(row['docketnumber']):
            docket_query = """
            MERGE (d:Docket {id: $docketnumber})
            WITH d
            MATCH (c:Case {id: $id})
            MERGE (c)-[:HAS_DOCKET]->(d)
            """
            await session.run(docket_query, {
                'id': str(row['id']),
                'docketnumber': str(row['docketnumber'])
            })

async def load_data_to_neo4j(df: pd.DataFrame):
    url = userdata.get('NEO4J_URL')
    username = "neo4j"
    password = userdata.get('NEO4J_PASSWORD')

    loader = AsyncNeo4jLoader(url, username, password)
    try:
        await loader.test_connection()
        await loader.clear_database()
        await loader.create_constraints()
        await loader.load_data(df)
        logger.info("Neo4j database updated successfully!")
    except Exception as e:
        logger.error(f"Error updating Neo4j database: {str(e)}")
    finally:
        await loader.close()

async def load_batch(batch, vector_store, splitter):
    loaded_documents = [Document(text=doc) for doc in batch]
    nodes = splitter.get_nodes_from_documents(loaded_documents)
    vector_index = VectorStoreIndex(nodes, storage_context=StorageContext.from_defaults(vector_store=vector_store))
    vector_index.storage_context.persist()

async def load_data_to_qdrant(documents: List, batch_size=1000, max_workers=4):
        qdrant_client = QdrantClient(url=userdata.get("QDRANT_URL"), api_key=userdata.get("QDRANT_API_KEY"))
        vector_store = QdrantVectorStore(client=qdrant_client, collection_name="law_docs")
        splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=100)

        batches = [documents[i:i+batch_size] for i in range(0, len(documents), batch_size)]

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            await asyncio.gather(*[load_batch(batch, vector_store, splitter) for batch in batches])

        logging.info("Qdrant database update completed")

async def main():
    llm = Groq(model="llama3-70b-8192", api_key=userdata.get('GROQ_API_KEY'), temperature=0)
    embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
    Settings.embed_model = embed_model
    Settings.chunk_size = 1024
    # Settings.llm = llm
    await load_data_to_neo4j(dataset_nodes)
    await load_data_to_qdrant(aggregated_df['case_opinions'].to_list())

# Use this for Google Colab
if __name__ == "__main__":
    from IPython import get_ipython
    ipython = get_ipython()
    if ipython is not None:
        ipython.run_line_magic('matplotlib', 'inline')
        ipython.run_line_magic('autoawait', 'on')
        await main()
    else:
        asyncio.run(main())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]